In [2]:
import pandas as pd
import numpy as np

# location processing
# from geotext import GeoText
from geopy import geocoders  
# import geocoder
# import requests

import multiprocessing as mp
from multiprocessing import Pool, Array, Lock
from time import time
from joblib import delayed, Parallel

In [3]:
all_data = pd.read_csv('../complete_data/complete_data_sentiment.csv')
all_data.head(1)

,tweet_id,location,date,likes,retweets,text,index,city,state,is_state,...,Total_Test_Results,Case_Fatality_Ratio,Testing_Rate,People_Tested,Mortality_Rate,polarity_neg,polarity_pos,polarity_compound,polarity_classif,polarity_classif2
0,1.250000e+18,"Pomona, CA",4/4/2020,89921,34665,A day without Hispanics .... this country coul...,1,Pomona,California,False,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0


In [15]:
num_unique_loc = len(unique_locs)

In [20]:
# returns coordinates given location
def get_coords2(i,loc,gn):
    if i % 100 == 0:
        print('location',i,':',loc,'percent:',round(i/num_unique_loc*100,2))
    try:
        location = gn.geocode(loc,exactly_one=False)
        coords = location[0][1]
        # print(coords)
    except:
        coords = (None,None)
    return coords

In [21]:
pool = mp.Pool()
# NUM_CPUS=8
gn = geocoders.GeoNames(username='tylerwu')
unique_locs = all_data.location.unique()
index = range(0,len(unique_locs))
# index[:3]
# coord_dict = [pool.apply(get_coords2, args=(i, unique_locs[i], gn)) for i in index[:32]] # mp package
# coord_dict = Parallel(n_jobs=NUM_CPUS)(delayed(get_coords2)(i,unique_locs[i],gn) for i in index[:10]) #joblib package
coord_dict = {unique_locs[i]:get_coords2(i,unique_locs[i], gn) for i in index} # old-fashioned
# # pool.close()
# coord_dict

location 0 : Pomona, CA percent: 0.0
location 100 : Springfield, MO percent: 0.72
location 200 : Santa Cruz, CA percent: 1.44
location 300 : Kissimmee, FL percent: 2.17
location 400 : Olympia, WA percent: 2.89
location 500 : El Cajon, CA percent: 3.61
location 600 : Salinas, California percent: 4.33
location 700 : Richmond, IN percent: 5.05
location 800 : Columbia, South Carolina, USA percent: 5.78
location 900 : Ocoee, FL percent: 6.5
location 1000 : New City, NY percent: 7.22
location 1100 : Monterey, California percent: 7.94
location 1200 : North Logan, UT percent: 8.66
location 1300 : Winchester, VA percent: 9.38
location 1400 : Auburn, Alabama percent: 10.11
location 1500 : Peoria, IL percent: 10.83
location 1600 : Colorado, US percent: 11.55
location 1700 : Joint Base Lewis-McChord, WA percent: 12.27
location 1800 : Clayton, NC percent: 12.99
location 1900 : Cocoa, FL percent: 13.72
location 2000 : Lawrence, MA percent: 14.44
location 2100 : Inwood, NY percent: 15.16
location 220

In [24]:
# sub_data = all_data.head()
coord_dict_lat = {coord:coord_dict[coord][0] for coord in coord_dict}
coord_dict_long = {coord:coord_dict[coord][1] for coord in coord_dict}
# coord_dict_lat
all_data['latitude'] = sub_data['location'].map(coord_dict_lat)
all_data['longitude'] = sub_data['location'].map(coord_dict_long)
all_data.head()

,tweet_id,location,date,likes,retweets,text,index,city,state,is_state,...,Testing_Rate,People_Tested,Mortality_Rate,polarity_neg,polarity_pos,polarity_compound,polarity_classif,polarity_classif2,latitude,longitude
0,1.250000e+18,"Pomona, CA",4/4/2020,89921,34665,A day without Hispanics .... this country coul...,1,Pomona,California,False,...,NaN,NaN,NaN,0.000,0.000,0.0000,0,0,34.05529,-117.75228
1,1.250000e+18,"Santa Barbara, CA",4/21/2020,34308,5245,XO face masks are back to raise money and Iâ...,2,Santa Barbara,California,False,...,765.392916,300100.0,3.607815,0.067,0.080,0.1280,1,1,34.42083,-119.69819
2,1.250000e+18,"California, USA",4/21/2020,0,0,@ImKingGinger Well aren't you special? My rela...,3,Not City,California,True,...,765.392916,300100.0,3.607815,0.258,0.052,-0.8104,-1,-1,34.68743,-116.78467
3,1.250000e+18,"Arizona, USA",4/19/2020,4,0,@seanhannity If I was to stay home and eat ice...,4,Not City,Arizona,True,...,728.012918,65652.0,3.729982,0.162,0.110,-0.4497,-1,-1,34.50030,-111.50098
4,1.250000e+18,"Medford, MA",4/6/2020,0,0,"@MisagaLion @tedpanos Yup, CT. Governor spoke ...",5,Medford,Massachusetts,False,...,NaN,NaN,NaN,0.224,0.108,-0.6988,-1,-1,42.41843,-71.10616


In [26]:
all_data.to_csv('../complete_data/complete_data.csv',index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'complete_data/complete_data.csv'